In [30]:
import psycopg2
import geoplotlib
from geoplotlib.utils import BoundingBox, DataAccessObject
from geoplotlib.colors import ColorMap
from pandas import DataFrame
import json
import pandas as pd

# Replace with your database credentials
db_connection = psycopg2.connect(
    user="tester",
    password="1234",
    host="localhost",
    port="5432",
    database="ThesisTry1"
)

#updated query to add anzahlmodule to extract Q
query = """
SELECT 
    we."Landkreis",
    SUBSTRING(we."Gemeindeschluessel" FROM 1 FOR 2) AS Kreisnummer,
    we."Bruttoleistung", we."AnzahlModule"
FROM 
    solar_extended As we
WHERE
    SUBSTRING(CAST(we."Registrierungsdatum" AS TEXT) FROM 1 FOR 4) = '2022'
GROUP BY 
    we."Landkreis", SUBSTRING(we."Gemeindeschluessel" FROM 1 FOR 2), we."Bruttoleistung", we."AnzahlModule";
"""
# Create a cursor and execute the query
cursor = db_connection.cursor()
cursor.execute(query)

# Fetch all the rows from the result
results = cursor.fetchall()

kreisliste = pd.DataFrame(results)
kreisliste.columns = ['landkreis','kreisnummer','Bruttoleistung', 'AnzahlModule']

# Drop rows with missing longitude or latitude values
kreisliste = kreisliste.dropna()


In [31]:
#kreistliste = kreisliste.drop_duplicates(subset="kreisnummer")
kreisliste['q'] = (kreisliste['Bruttoleistung'] / kreisliste['AnzahlModule']) * 1000
display(kreisliste)

,landkreis,kreisnummer,Bruttoleistung,AnzahlModule,q
0,Ahrweiler,07,0,1,285
1,Ahrweiler,07,0,1,300
2,Ahrweiler,07,0,2,150
3,Ahrweiler,07,0,3,100
4,Ahrweiler,07,0,4,75
...,...,...,...,...,...
194040,Zwickau,14,462,100,4618
194041,Zwickau,14,1196,2719,440
194042,Zwickau,14,3019,8123,372
194043,Zwickau,14,5120,11130,460


In [55]:
#Without_outliers
# Create a new DataFrame containing rows where 'q' is not between 300 and 500
filtered_df = kreisliste[(kreisliste['q'] < 800) & (kreisliste['q'] > 200)].copy()
display(filtered_df)
#display(kreisliste)

,landkreis,kreisnummer,Bruttoleistung,AnzahlModule,q
0,Ahrweiler,7,0,1,285
1,Ahrweiler,7,0,1,300
5,Ahrweiler,7,0,1,340
6,Ahrweiler,7,0,1,360
7,Ahrweiler,7,0,1,370
...,...,...,...,...,...
194039,Zwickau,14,320,864,370
194041,Zwickau,14,1196,2719,440
194042,Zwickau,14,3019,8123,372
194043,Zwickau,14,5120,11130,460


In [56]:
import numpy as np
import pandas as pd

# Group by 'Landkreis' and 'kreisnummer', then calculate the sum of 'brutoleistung'
sum_bruttoleistung_per_kreisnummer = filtered_df.groupby(['kreisnummer'])['Bruttoleistung'].sum().reset_index()
#sum_bruttoleistung_per_kreisnummer = kreisliste.groupby(['kreisnummer'])['Bruttoleistung'].sum().reset_index()
# Print the new DataFrame

#pd.options.display.float_format = '{:.0f}'.format

#display(sum_bruttoleistung_per_kreisnummer)

# Convert 'kreisnummer' column to integer type
filtered_df['kreisnummer'] = filtered_df['kreisnummer'].astype(int)

# Dictionary mapping kreisnummer to Bundesland names
kreis_bundesland_mapping = {
    1: 'Schleswig Holstein',
    2: 'Hamburg',
    3: 'Lower Saxony',
    4: 'Bremen',
    5: 'North-Rhine-Westphalia',
    6: 'Hesse',
    7: 'Rhineland Palatinate',
    8: 'Baden-Württemberg',
    9: 'Bayern',
    10: 'Saarland',
    11: 'Berlin',
    12: 'Brandenburg',
    13: 'Mecklenburg-Vorpommern',
    14: 'Saxony',
    15: 'Saxony-Anhalt',
    16: 'Thuringia'
}

# Check unique values in the 'kreisnummer' column
unique_kreisnummern = filtered_df['kreisnummer'].unique()
print("Unique Kreisnummern:", unique_kreisnummern)

# Rename 'Kreisnummer' column to 'Bundesland Number'
sum_bruttoleistung_per_kreisnummer.rename(columns={'kreisnummer': 'Bundesland Number'}, inplace=True)

# Add Bundesland column based on kreisnummer
sum_bruttoleistung_per_kreisnummer['Bundesland'] = sum_bruttoleistung_per_kreisnummer['Bundesland Number'].map(kreis_bundesland_mapping)

# Setting display format for float values
pd.options.display.float_format = '{:.0f}'.format

# Display the DataFrame
display(sum_bruttoleistung_per_kreisnummer)

Unique Kreisnummern: [ 7  9  8 16 15  3 12 14  6 11  5  4  1  2 13 10]


,Bundesland Number,Bruttoleistung,Bundesland
0,1,253545,Schleswig Holstein
1,2,9284,Hamburg
2,3,426827,Lower Saxony
3,4,7571,Bremen
4,5,660547,North-Rhine-Westphalia
5,6,296805,Hesse
6,7,362326,Rhineland Palatinate
7,8,623836,Baden-Württemberg
8,9,2065379,Bayern
9,10,79443,Saarland


In [57]:
#Kreisnummern so verändern, dass sie mit den GoJson Nummern entsprechen
sum_bruttoleistung_per_kreisnummer = sum_bruttoleistung_per_kreisnummer.astype({'kreisnummer' : 'float'})       #get rid of 0 in front
sum_bruttoleistung_per_kreisnummer = sum_bruttoleistung_per_kreisnummer.astype({'kreisnummer' : 'int'})       #get rid of 0 in front
sum_bruttoleistung_per_kreisnummer = sum_bruttoleistung_per_kreisnummer.astype({'kreisnummer' : 'str'})       #only strings can be verglichen werden to show results
sum_bruttoleistung_per_kreisnummer['kreisnummer'] = sum_bruttoleistung_per_kreisnummer['kreisnummer'] + '000'         #add 000
display(sum_bruttoleistung_per_kreisnummer)

KeyError: "Only a column name can be used for the key in a dtype mappings argument. 'kreisnummer' not found in columns."

In [54]:
#This cell shows the data after validating. Filtering out Q only between 200 to 500


print("Minimum Installed Power: ", sum_bruttoleistung_per_kreisnummer['InstalledPower'].min())
sum_bruttoleistung_per_kreisnummer['kreisnummer'].to_csv('dbdata.csv', index=False)
with open('website/LandkreiseALL.json',encoding='utf8') as handle:
    country_geo = json.loads(handle.read())

#erstellen eines neuen DF für die Landkreisliste 
struktur = {'kreisnummer' : []}
kreislist = DataFrame(struktur)

#beschreiben der kreisliste mit entsprechenden Namen aus der JSON Datei
j = 0
for i in country_geo['features']:
    kreislist.at[j,'kreisnummer'] =  i['properties']['SN_KRG']      #zuweisung vom Kenncode -- dividieren durch 1000 weil Stammdaten aus JSON aus irgendeinem Grund drei Nullen angehängt haben
    j = j+1
#kreislist.to_csv('jsoncompare.csv', index=False)    
kreislist['kreisnummer'].to_csv('geojsonkreisnummer.csv', index=False)

# Merge based on the "landkreis" column and select the desired columns
merged_df = DataFrame.merge(kreislist, sum_bruttoleistung_per_kreisnummer, on='kreisnummer', how='outer')
merged_df['InstalledPower'].fillna(0, inplace=True) #fillna function puts o instead of Null to a district. 
merged_df = merged_df.drop_duplicates(subset=["kreisnummer"])
merged_df['InstalledPower'].round(2) # Todo: For some reason it is not working. Get back to it later. 
merged_df['InstalledPower'].to_csv('mergedcsvkreisnummer.csv', index=False)
#print(DataFrame.merge([kreisliste,merged_df]).drop_duplicates(keep=False))
colorStep =  [ 0.6, 100, 250, 500, 1000, 5000, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000,100000, 200000, 300000, 400000, 500000, 1000000]


map = getKarte(geoDaten = country_geo ,kreisliste = merged_df, colorstep= colorStep, key_on = 'InstalledPower')

map

KeyError: 'InstalledPower'